<a href="https://colab.research.google.com/github/Preyansh17/Pytorch/blob/master/Regression_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credits to @knighterudite https://github.com/knighterudite/ML101/blob/master/dnn_regression.ipynb

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [2]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv")
df.drop(columns=['date'], inplace=True)
df.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
columns = df.columns
num_columns = df._get_numeric_data().columns
print("Numeric Data: {}".format(num_columns))
print("Categorical Data: {}\n".format(list(set(columns) - set(num_columns)))) 
#check for missing values
print("Missing value check -")
df.isnull().sum() #no missing values

Numeric Data: Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
       'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')
Categorical Data: []

Missing value check -


Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

We can see that only numeric data is there in our dataset and no missing values in any of the columns.

In [4]:
X = df.iloc[:,1:] 
Y = df.iloc[:,0]
X.head()

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


X is our input feature matrix. Y is our target vector with number of Appliances for each training example. 

In [5]:
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size= 0.2, random_state=0)

We use sklearn's train_test_split for a 80:20 training:test split.

In [6]:
scaler = RobustScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train

array([[ 0.        ,  0.27439547, -0.65535401, ..., -0.15835777,
         0.79964757,  0.79964757],
       [20.        , -0.54879094, -0.24634289, ..., -0.65102639,
         0.55622331,  0.55622331],
       [ 0.        , -0.93843252, -0.47981276, ..., -0.85630499,
        -0.84273881, -0.84273881],
       ...,
       [ 0.        ,  0.03658606, -0.62200117, ..., -0.23753666,
         0.20176787,  0.20176787],
       [ 0.        ,  0.15914937,  0.3978935 , ...,  0.85630499,
         0.55718923,  0.55718923],
       [ 0.        , -0.38964157,  0.72440023, ..., -0.04692082,
        -0.72816387, -0.72816387]])

We again used sklearn.preprocessing for Standardising our Dataset

In [7]:
X_train = (torch.Tensor(X_train))
y_train = (torch.Tensor(np.array(y_train)))
X_test = (torch.Tensor(X_test))
y_test = (torch.Tensor(np.array(y_test)))

Converted numpy arrays to torch tensor.

In [8]:
model = nn.Sequential(nn.Linear(X_train.shape[1],100),
                      nn.ReLU(),
                      nn.Linear(100,75),
                      nn.ReLU(),
                      nn.Linear(75,50),
                      nn.ReLU(),
                      nn.Linear(50,25),
                      nn.ReLU(),
                      nn.Linear(25,1))
model

Sequential(
  (0): Linear(in_features=27, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=75, bias=True)
  (3): ReLU()
  (4): Linear(in_features=75, out_features=50, bias=True)
  (5): ReLU()
  (6): Linear(in_features=50, out_features=25, bias=True)
  (7): ReLU()
  (8): Linear(in_features=25, out_features=1, bias=True)
)

In [9]:
loss_func = torch.nn.MSELoss()
loss_func

MSELoss()

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [11]:
epochs = 850
loss_list = []
print("Training..")
for i in range(epochs):
  y_pred = model(X_train).squeeze()
  loss = loss_func(y_pred, y_train)
  loss_list.append(loss.item())
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if(i%100 == 0):
    print("Epoch: {}, Loss: {}".format(i, loss.item()))

Training..
Epoch: 0, Loss: 19775.49609375
Epoch: 100, Loss: 10041.552734375
Epoch: 200, Loss: 8878.2080078125
Epoch: 300, Loss: 8255.63671875
Epoch: 400, Loss: 7828.84375
Epoch: 500, Loss: 7312.8232421875
Epoch: 600, Loss: 6487.759765625
Epoch: 700, Loss: 5742.81591796875
Epoch: 800, Loss: 5131.5576171875


In [12]:
loss_test = loss_func((model(X_test)).squeeze(), y_test)
loss_test.item()

7686.5810546875